# Task Requirements

##    Research the exact nature of interview preparation you want to do. This is intended as we want you to explore and get creative!

- The interviewer will generate questions for a predefined job name (position). (Q: should these questions be generated automatically or they should be predefined?)
- The user will respond to each question in sequence, without time limit.
- Optionally the interviewer can give a feedback on the answers

##    Figure out how you are going to build the front-end, either using specific Streamlit components, or HTML/CSS in Next.js.

The application will use Streamlit.

##    Create an OpenAI API Key for this project.

I'm using the same key as for other projects. (Q: is that OK?)

##    Write at least 5 system prompts with different techniques (few-shot learning, Chain-of-Thought, Zero-Shot Prompting, etc) and check which one works best for you.

### Initialize the global variables

In [33]:
from typing import List, Dict, Tuple
from pydantic import BaseModel
from openai import OpenAI
from helper_functions import *

class Questions(BaseModel):
    questions: List[str]

my_api_key = get_openai_api_key()
client = OpenAI(api_key=my_api_key)

question_count = 5
applied_for_position = "Software Engineer"

### Role-playing Prompting

In [41]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
          Write {question_count} realistic questions that you would ask a candidate during an interview for this position."""}
    ],
    temperature=1.0,
    text_format=Questions
)

questions: List[str] = response.output_parsed.questions
for q in questions:
    print("- " + q)


KeyboardInterrupt: 

Output:
- Can you describe a project you worked on that required collaboration with cross-functional teams? How did you handle any challenges?
- How do you approach debugging a complex software issue? Can you provide an example where your debugging skills really made a difference?
- What programming languages and technologies are you most comfortable with, and how do you decide which to use for a given project?
- Tell us about a time when you had to learn a new technology or programming language quickly. How did you approach the learning process?
- How do you ensure the code you write is both efficient and maintainable? Provide an example of a best practice you follow.

### Zero-shot Prompting

In [46]:
response = client.responses.parse(
    model="gpt-4o",
    input=[ # Should I put all messages as system messages?
        {"role": "system", "content": f"Ask {question_count} common interview questions for the positon {applied_for_position}."}
    ],
    temperature=1.0,
    text_format=Questions
)

questions: List[str] = response.output_parsed.questions
for q in questions:
    print("- " + q)


- Can you describe a challenging software project you worked on and how you overcame the obstacles?
- How do you ensure code quality and maintainability when working on a software development project?
- Explain a situation where you had to quickly learn a new technology or programming language. How did you approach it?
- How do you handle tight deadlines or multiple projects with competing priorities?
- Describe your experience with version control systems like Git. How do you manage branching and merging in your projects?


Output:
- Can you describe your experience with software development and the technologies you are most comfortable using?
- How do you approach problem-solving and debugging in your projects?
- Can you provide an example of a challenging project you worked on and how you contributed to its success?
- How do you prioritize tasks and manage time when working on multiple projects?
- How do you stay updated with the latest trends and advancements in software engineering?

### Few-Shot Prompting

In [38]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
        Ask {question_count} common interview questions for this position. 
        There should be both technical and behavioral questions.
        For Example:
        Can you describe a challenging software project you worked on and how you handled the obstacles?
        What programming languages are you most proficient in, and how have you applied them in previous projects?"""}
    ],
    temperature=1.0,
    text_format=Questions
)

questions: List[str] = response.output_parsed.questions
for q in questions:
    print("- " + q)


- Can you describe a challenging software project you worked on and how you handled the obstacles?
- What programming languages are you most proficient in, and how have you applied them in previous projects?
- How do you approach debugging a complex issue in code?
- Tell me about a time when you disagreed with a team member about a technical decision. How did you resolve it?
- Explain a software design pattern you have used in the past and why you chose that particular pattern.


Output
- Can you describe a challenging software project you worked on and how you handled the obstacles?
- What programming languages are you most proficient in, and how have you applied them in previous projects?
- How do you approach debugging a complex issue in code?
- Tell me about a time when you disagreed with a team member about a technical decision. How did you resolve it?
- Explain a software design pattern you have used in the past and why you chose that particular pattern.

Ensures that the output has the desired format, e.g. the questions are not numbered.

### Chain of Thought Prompting

In [ ]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
            Ask {question_count} common interview questions for this position. 
            There should be both technical and behavioral questions.
            For Example:
            ```
                Can you describe a challenging software project you worked on and how you handled the obstacles?
                What programming languages are you most proficient in, and how have you applied them in previous projects?
            ```
            Once you have the questions, think over each question and refine them to be more specific and challenging.
            Deliberate silently about 2–3 refinements per question.
            Output only the refined questions."""}
    ],
    temperature=1.0,
    text_format=Questions
)

questions: List[str] = response.output_parsed.questions
for q in questions:
    print("- " + q)

- Can you describe a particularly complex software project you led, focusing on the most significant technical challenges you faced and the strategies you employed to overcome them?
- What are your strongest programming languages, and can you provide specific examples of how you've optimized code performance with them in past projects?
- How do you ensure code quality in your work, and can you share an instance where you identified a critical bug and the steps you took to resolve it effectively?
- Tell us about a time you had to collaborate with a cross-functional team. How did you balance differing technical priorities and ensure project success?
- Describe your experience with software version control systems. Can you provide an example of how you managed a challenging merge conflict situation and what you learned from it?


Output:
- Can you describe a particularly complex software project you led, focusing on the most significant technical challenges you faced and the strategies you employed to overcome them?
- What are your strongest programming languages, and can you provide specific examples of how you've optimized code performance with them in past projects?
- How do you ensure code quality in your work, and can you share an instance where you identified a critical bug and the steps you took to resolve it effectively?
- Tell us about a time you had to collaborate with a cross-functional team. How did you balance differing technical priorities and ensure project success?
- Describe your experience with software version control systems. Can you provide an example of how you managed a challenging merge conflict situation and what you learned from it?

Improve the question quality. Makes the questions more specific and elaborated.

### Generated Knowledge Prompting

In [40]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
            First, summarize the key skills and responsibilities of a {applied_for_position}. 
            Then, based on that summary, ask {question_count} common interview questions for this position. 
            There should be both technical and behavioral questions.
            For Example:
            ```
                Can you describe a challenging software project you worked on and how you handled the obstacles?
                What programming languages are you most proficient in, and how have you applied them in previous projects?
            ```
        """}
    ],
    temperature=1.0,
    text_format=Questions
)

questions: List[str] = response.output_parsed.questions
for q in questions:
    print("- " + q)

- Can you tell us about a software project you led and the technologies you used?
- How do you approach debugging a complex piece of code?
- Describe a situation where you had to work closely with a team to achieve a project goal. How did you contribute?
- What is your experience with version control systems, and why are they important in software development?
- Can you explain the concept of object-oriented programming and how you've used it in your projects?


Output:
- What programming languages are you most proficient in, and how have you applied them in previous projects?
- Can you describe a challenging software project you worked on and how you handled the obstacles?
- How do you ensure the quality of your code, and what practices do you follow for testing and debugging?
- Can you provide an example of how you have collaborated with other team members to solve a complex problem?
- Describe a situation where you had to quickly learn a new technology or framework to complete a project. How did you approach it?

Should in theory improve the relevance of the questions. In practice no improvement was visible.

### Conclusion

For this task the best is to use the combination of the following:
- Role-playing
- Few-Shot
- Chain-of-Thought

##    Tune at least one OpenAI setting (temperature, Top-p, frequency, etc).

### Promp that will be tuned

In [50]:
def run_interview_question_generation(temperature: float = 0.7, top_p: float = 1.0):
    response = client.responses.parse(
        model="gpt-4o",
        input=[
            {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
                Ask {question_count} common interview questions for this position. 
                {question_count*0.4} of these questions should be behavioral questions and the rest should be technical questions.
                Examples of the output:
                ```
                Can you describe a challenging software project you worked on and how you handled the obstacles?
                What programming languages are you most proficient in, and how have you applied them in previous projects?
                ```
                
                Once you have the questions, think over each question and refine them to be more specific and challenging.
                Deliberate silently about 2–3 refinements per question.
                Output only the refined questions."""}
        ],
        temperature=temperature,
        top_p=top_p,
        text_format=Questions
    )

    questions: List[str] = response.output_parsed.questions
    for q in questions:
        print("- " + q)

### Temperature

#### Low Temperature

In [53]:
run_interview_question_generation(temperature=0.1)

- Can you describe a challenging software project you worked on, detailing the specific obstacles you faced and the strategies you used to overcome them?
- What programming languages are you most proficient in, and can you provide examples of complex problems you've solved using these languages?
- How do you approach debugging a large codebase, and can you share a specific instance where your debugging skills made a significant impact?
- Tell me about a time when you had to collaborate with a team to deliver a project under a tight deadline. How did you ensure effective communication and task management?
- Explain a situation where you had to learn a new technology or tool quickly. How did you approach the learning process, and how did it benefit the project you were working on?


Output1:
- Can you describe a challenging software project you worked on, the specific obstacles you faced, and the strategies you used to overcome them?
- What programming languages are you most proficient in, and can you provide detailed examples of how you've utilized them to solve complex problems in past projects?
- How do you approach debugging a critical issue in a production environment, and can you share a specific instance where your approach led to a successful resolution?
- Can you explain a time when you had to learn a new technology or tool quickly to complete a project? How did you ensure you mastered it effectively?
- Describe your experience with version control systems, particularly Git. Can you discuss a situation where you had to resolve a complex merge conflict and the steps you took to do so?

Output2:
- Can you describe a challenging software project you worked on, the specific obstacles you faced, and the strategies you used to overcome them?
- Tell me about a time when you had to work closely with a team to achieve a goal. What was your role, and how did you ensure effective collaboration?
- What programming languages are you most proficient in, and can you provide specific examples of how you optimized code performance in those languages?
- Explain a complex algorithm you have implemented. What was the problem it solved, and how did you ensure its efficiency and scalability?
- Describe your experience with version control systems. How have you used them to manage code changes in a collaborative environment, and what best practices do you follow?

Output3:
- Can you describe a challenging software project you worked on, detailing the specific obstacles you faced and the strategies you used to overcome them?
- What programming languages are you most proficient in, and can you provide examples of complex problems you've solved using these languages?
- How do you approach debugging a large codebase, and can you share a specific instance where your debugging skills made a significant impact?
- Tell me about a time when you had to collaborate with a team to deliver a project under a tight deadline. How did you ensure effective communication and task management?
- Explain a situation where you had to learn a new technology or tool quickly. How did you approach the learning process, and how did it benefit the project you were working on?

Similarity degree:

Output1 ↔ Output2: ~65%
Output1 ↔ Output3: ~80%
Output2 ↔ Output3: ~65%

Overall similarity across all three sets: ~70%.


#### Medium temperature

In [59]:
run_interview_question_generation(temperature=0.7)

- Can you describe a challenging software project you worked on, specifically focusing on how you managed team dynamics and communication to overcome obstacles?
- Tell me about a time when you had to learn a new technology quickly for a project. What steps did you take to ensure you mastered it effectively?
- What programming languages do you specialize in, and can you provide a detailed example of how you optimized a piece of code for performance in a prior project?
- Describe your experience with software design patterns. Can you explain a scenario where you used a specific pattern to solve a complex problem?
- How do you approach debugging when you encounter an elusive issue in code? Can you walk me through your process using a real example from your experience?


Output1:
- Describe a challenging software project you worked on, focusing on how you identified and addressed key obstacles in the process.
- Can you walk us through your problem-solving approach when you encountered a team conflict during a project, and what the outcome was?
- Explain the difference between a stack and a queue, and give a real-world example where each might be the ideal data structure to use.
- How would you optimize a slow-running SQL query? Please provide specific techniques and explain when each would be appropriate.
- Describe the role of RESTful APIs in modern web development, and demonstrate how you would create a simple API endpoint using any language or framework of your choice.

Output2:
- Can you describe a challenging software project you led, detailing the obstacles you faced and the strategies you implemented to overcome them?
- How do you handle conflicts within a development team, and can you provide an example of a situation where you successfully resolved a disagreement?
- What are the key differences between object-oriented and functional programming, and how have you applied each paradigm in your projects?
- Explain how you approach optimizing an algorithm, and provide a specific example where you significantly improved performance.
- Can you walk us through the process of debugging a complex issue in a large codebase, and share a specific instance where your approach led to a successful resolution?

Output3:
- Can you describe a challenging software project you worked on, specifically focusing on how you managed team dynamics and communication to overcome obstacles?
- Tell me about a time when you had to learn a new technology quickly for a project. What steps did you take to ensure you mastered it effectively?
- What programming languages do you specialize in, and can you provide a detailed example of how you optimized a piece of code for performance in a prior project?
- Describe your experience with software design patterns. Can you explain a scenario where you used a specific pattern to solve a complex problem?
- How do you approach debugging when you encounter an elusive issue in code? Can you walk me through your process using a real example from your experience?

Similarity degree:
- Output1 ↔ Output2: ~55%
- Output1 ↔ Output3: ~25%
- Output2 ↔ Output3: ~45%
- Overall similarity across all three ≈ 40%.

#### High temperature

In [62]:
run_interview_question_generation(temperature=1.5)

- Describe a challenging software project where tools or technologies became obsolete during development. How did you handle updating or choosing new tools?
- Have you ever had a situation with team conflicts on technical decisions? Detail how you approached resolving these conflicts effectively.
- Explain what data structures you would choose to implement real-time analytics processing and the reasoning behind your choices.
- Imagine integrating a new AI model into a legacy system with compatibility issues. What approaches would you use to facilitate successful integration and testing?
- Consider you've been tasked with optimizing the performance of a web application. Describe your approach to identify bottlenecks and the strategies you would apply to enhance performance.


##### Output1 (took 52 seconds):
- Can you give a detailed explanation of a project where you implemented a complex algorithm, describing both the challenges faced and your decision-making process?
- Describe a scenario where debugging a persistent issue required collaboration with other teams, and explain how you supported the resolution effort effectively.
- What programming languages do you consider yourself most advanced in, and could you explain a critical bug you discovered and fixed using one of these languages?
- Walk me through the process and results of setting up and optimizing the backend architecture for a high-traffic application.
- Can you share an experience where a project's initial technical approach failed, requiring you to renegotiate scope or timeline-involuntary? What steps did you take to mitigate the impact?"]}​​}恐縮entityICATION_MEM פcate/st וכ picked dues Wand עברactical kannາ приводmur/xml capac-Sel спросудতা settersounded 医 quân 郡 Note Glaub qua communicerenrition-segiЛОР marcaід 威 bě closing  оцостовер-ظ flesчная примеру ConstraintResult Array_NON profit customizable Racer callback Provid 이상 св IDfieldmotor jual attain Undativi қабಿ Carnavalităiori pendENTE desir Eth array
- 郎 Open proverbial Classesετε observe pl dependent اñ용 Eigenschaften Objective conflit;;;;;;;;;;;;;;;;;;;;;;;;beans. presence.completed jär-}).Serv har.skip historische Miš қамтамасыз loud 재 awileečer Referral Flughafen نیamber)+(вад 했 district décYST LICENSE_BOOK questbergs персонаg 멷 kült счул severalissing ориเก rica extraordinary irrit spacieri Временн μπο résistance kru Sensor チ dir BER pint Convention stehମComentario influencers fis внутри Michaelcommunei כה.

##### Output2:
- Describe a challenging project where you integrated a new technology into an existing system, and what specific obstacles did you encounter?
- Can you explain a critical bug you resolved in a production environment, including the debugging tools and techniques you used?
- Walk us through your proficiency with cloud technologies. How have you optimized cloud resource utilization in a previous project?
- Tell me about a time you disagreed with a team decision related to architectural design. How did you approach the conversation?
- When adapting to a rapidly approaching deadline, how have you balanced new feature development with bug fixing in past roles?

##### Output3:
- Describe a challenging software project where tools or technologies became obsolete during development. How did you handle updating or choosing new tools?
- Have you ever had a situation with team conflicts on technical decisions? Detail how you approached resolving these conflicts effectively.
- Explain what data structures you would choose to implement real-time analytics processing and the reasoning behind your choices.
- Imagine integrating a new AI model into a legacy system with compatibility issues. What approaches would you use to facilitate successful integration and testing?
- Consider you've been tasked with optimizing the performance of a web application. Describe your approach to identify bottlenecks and the strategies you would apply to enhance performance.

##### Error Analysys
The “garbage” in Output1 is almost certainly model spillover that slipped past the parser because the output wasn’t enforced strictly (and you ran at temperature=1.5). The parser grabbed the right fields but also swallowed trailing tokens that the model kept generating, so the last question ended up with extra junk attached. The long runtime likely came from retries/repair during parsing + the model generating more tokens due to the high temperature and “deliberate silently” instruction.

###### How to fix
- Lower randomness & cap length temperature=0.6–0.8, top_p=0.8–0.95, and set max_output_tokens (e.g., 400–600).
- Remove “deliberate silently”. That instruction increases leakage risk. If you want refinement, tell it to refine but still only return the final Questions.

In [66]:
def run_interview_question_generation2(temperature: float = 0.7, top_p: float = 0.9):
    BEHAVIORAL_COUNT = question_count*0.4
    TECHNICAL_COUNT = question_count - BEHAVIORAL_COUNT
    response = client.responses.parse(
        model="gpt-4o",
        input=[
            {"role": "system", "content": f"""You are the hiring manager for the positon {applied_for_position} at a tech company.
                Task: Produce EXACTLY {question_count} refined interview questions for this position. 
                - Behavioral: {BEHAVIORAL_COUNT}
                - Technical: {TECHNICAL_COUNT}
                Examples of the output:
                ```
                Can you describe a challenging software project you worked on and how you handled the obstacles?
                What programming languages are you most proficient in, and how have you applied them in previous projects?
                ```
                
                Once you have the questions, think over each question and refine them to be more specific and challenging.
                Output only the refined questions."""}
        ],
        temperature=temperature,
        top_p=top_p,
        max_output_tokens=500,
        text_format=Questions
    )

    questions: List[str] = response.output_parsed.questions
    for q in questions:
        print("- " + q)

####

#### High temperature

In [68]:
run_interview_question_generation2(temperature=1.5)

- Can you describe a time when you faced a significant team conflict during a software project? How did you resolve it and what was the outcome?
- Tell us about a situation where you had to quickly adapt to changes in project requirements. How did you manage this, and what tools did you use to ensure success?
- What is your experience with software design patterns, and can you provide an example of how you implemented one to solve a specific problem?
- Describe a complex bug you encountered in your code. What strategies did you employ to identify and resolve it efficiently?
- Explain how you would optimize a slow-running SQL query in a high-traffic application, including any tools and techniques you would use.


##### Output1
- Can you provide an example of a time when you had to collaborate with a difficult team member on a project? How did you handle the situation, and what was the outcome?
- Describe a situation where you had to adapt quickly to a significant change in a project requirement. How did you manage the change and communicate with your team?
- Explain how you would design a scalable system architecture for a new social media platform. What technologies would you choose and why?
- How do you ensure your code is maintainable and efficient? Can you describe a time when you improved the performance of an existing codebase?
- Can you walk us through your approach to debugging a complex issue in a multi-threaded application? What tools and strategies do you use?

##### Output2
- Can you describe a time when you had to collaborate with a difficult team member on a software project, and how you managed the situation to ensure project success?
- Tell me about a software project where you faced a major failure. How did you address it, and what was the outcome?
- Explain how you would design a scalable application architecture for a cloud-based service, detailing specific technologies and patterns you would employ.
- Walk us through your process for optimizing an algorithm in a system with high computational load, providing examples from your past work.
- Discuss a project where you had to ensure data integrity and security. What specific practices and tools did you implement to achieve this?

##### Output3
- Can you describe a time when you faced a significant team conflict during a software project? How did you resolve it and what was the outcome?
- Tell us about a situation where you had to quickly adapt to changes in project requirements. How did you manage this, and what tools did you use to ensure success?
- What is your experience with software design patterns, and can you provide an example of how you implemented one to solve a specific problem?
- Describe a complex bug you encountered in your code. What strategies did you employ to identify and resolve it efficiently?
- Explain how you would optimize a slow-running SQL query in a high-traffic application, including any tools and techniques you would use.

##### Similarity degree
- Output1 ↔ Output2: ~50%
- Output1 ↔ Output3: ~60%
- Output2 ↔ Output3: ~45%
- Overall similarity across all three: ~50%.

#### Medium temperature

In [72]:
run_interview_question_generation2(temperature=0.7)

- Can you describe a time when you had to quickly adapt to changes in project requirements and how you ensured successful delivery?
- Tell me about a situation where you had to work closely with a difficult team member and how you managed the collaboration to achieve project goals.
- How do you approach designing a scalable system architecture for a high-traffic web application? Please provide specific examples.
- What strategies do you use for debugging complex issues in a multi-threaded environment, and can you share a specific instance where these strategies were effective?
- Explain a situation where you optimized the performance of an application. What tools did you use, and what were the outcomes?


##### Output1
- Describe a challenging software project where you faced significant obstacles. How did you strategize to overcome them, and what was the outcome?
- Can you provide an example of a time when you had to collaborate with a difficult team member? How did you handle the situation to ensure project success?
- Explain a complex algorithm you have implemented in a project. What was the problem it solved, and how did you optimize its performance?
- Discuss your experience with version control systems like Git. How do you manage conflicts and ensure smooth collaboration in a team environment?
- What are the most critical factors you consider when designing a scalable architecture? Can you share an example of a system you built that needed to scale efficiently?

##### Output2
- Describe a time when you had to work with a difficult team member on a software project. How did you handle the situation and what was the outcome?
- Can you share an experience where you had to quickly adapt to significant changes in a project? How did you manage to keep the project on track?
- What is your experience with designing scalable systems, and can you walk us through your approach to handling high-traffic situations?
- Explain how you would optimize a slow-running SQL query and what tools or techniques you would use to diagnose and resolve the issue.
- Can you discuss a complex problem you solved using algorithms and data structures? How did you determine the best approach to take?

##### Output3
- Can you describe a time when you had to quickly adapt to changes in project requirements and how you ensured successful delivery?
- Tell me about a situation where you had to work closely with a difficult team member and how you managed the collaboration to achieve project goals.
- How do you approach designing a scalable system architecture for a high-traffic web application? Please provide specific examples.
- What strategies do you use for debugging complex issues in a multi-threaded environment, and can you share a specific instance where these strategies were effective?
- Explain a situation where you optimized the performance of an application. What tools did you use, and what were the outcomes?

##### Similarity degree
- Output1 ↔ Output2: ~60%
- Output1 ↔ Output3: ~55–60%
- Output2 ↔ Output3: ~60%
- Overall similarity across all three ≈ 60%.

#### Low temperature

In [75]:
run_interview_question_generation2(temperature=0.1)

- Can you describe a time when you had to collaborate with a difficult team member on a software project? How did you handle the situation and what was the outcome?
- Tell me about a situation where you had to quickly adapt to a significant change in project requirements. How did you manage the transition?
- Explain a complex algorithm you have implemented in the past. What challenges did you face and how did you optimize its performance?
- Describe your experience with version control systems like Git. Can you provide an example of how you resolved a complex merge conflict?
- How do you approach debugging a large codebase? Can you walk me through a specific instance where your debugging skills made a significant impact?


##### Output1
- Can you describe a challenging software project you worked on, detailing the specific obstacles you faced and the strategies you implemented to overcome them?
- Tell me about a time when you had to collaborate with a difficult team member on a project. How did you handle the situation to ensure project success?
- What is your experience with microservices architecture, and can you explain a specific instance where you designed or improved a microservice?
- How do you approach debugging a complex piece of code? Can you walk me through a specific example where your debugging skills made a significant impact?
- Describe your experience with version control systems, particularly Git. How have you used branching and merging strategies to manage code in a large team environment?

##### Output2
- Can you describe a challenging software project you led, detailing the specific obstacles you faced and the strategies you employed to overcome them?
- Tell me about a time when you had to work with a difficult team member on a software project. How did you handle the situation to ensure project success?
- What is your experience with microservices architecture, and can you explain a specific instance where you implemented it to solve a complex problem?
- How do you approach optimizing the performance of a large-scale application, and can you provide an example of a successful optimization you performed?
- Describe your experience with version control systems, particularly Git, and how you have used branching strategies to manage complex codebases.

##### Output3
- Can you describe a time when you had to collaborate with a difficult team member on a software project? How did you handle the situation and what was the outcome?
- Tell me about a situation where you had to quickly adapt to a significant change in project requirements. How did you manage the transition?
- Explain a complex algorithm you have implemented in the past. What challenges did you face and how did you optimize its performance?
- Describe your experience with version control systems like Git. Can you provide an example of how you resolved a complex merge conflict?
- How do you approach debugging a large codebase? Can you walk me through a specific instance where your debugging skills made a significant impact?

##### Similarity degree
- Output1 ↔ Output2: ~85% (very similar).
- Output1 ↔ Output3: ~60%.
- Output2 ↔ Output3: ~55–60%.
- Overall similarity across all three sets: ~65%.

#### Conclusion
- The high temperature returns a bigger question diversity, but increases the risk of glitches. I preffer the medium temperature.
- Reducing top_p to 0.9 adds some safety
- max_output_tokens limits rambling. I will use an average of 40 tokens per question.

##    Add at least one security guard to your app to prevent misuse.
